## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Beian,NO,63.6559,9.5683,52.00,87,75,2.30,broken clouds
1,1,Albany,US,42.6001,-73.9662,81.18,71,35,3.87,scattered clouds
2,2,Torbay,CA,47.6666,-52.7314,72.37,72,75,9.22,broken clouds
3,3,Ushuaia,AR,-54.8000,-68.3000,35.26,87,75,16.11,light rain
4,4,Yelanskiy,RU,56.8419,62.5014,55.78,93,20,6.04,few clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 85


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Albany,US,42.6001,-73.9662,81.18,71,35,3.87,scattered clouds
6,6,Kapaa,US,22.0752,-159.3190,84.52,67,20,14.97,few clouds
12,12,Gizo,SB,-8.1030,156.8419,83.79,69,2,6.04,clear sky
21,21,Hithadhoo,MV,-0.6000,73.0833,81.86,72,99,7.34,overcast clouds
26,26,Santa Cruz,BO,-17.8000,-63.1667,78.85,57,20,16.11,few clouds
29,29,Port Moresby,PG,-9.4431,147.1797,80.02,83,75,11.50,broken clouds
30,30,Katsuura,JP,35.1333,140.3000,81.18,74,0,19.71,clear sky
34,34,Vilhena,BR,-12.7406,-60.1458,80.82,50,40,2.30,scattered clouds
35,35,Manono,CD,-7.3000,27.4167,77.29,34,68,3.96,broken clouds
38,38,Jamestown,US,42.0970,-79.2353,80.24,53,0,11.50,clear sky


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                211
City                   211
Country                211
Lat                    211
Lng                    211
Max Temp               211
Humidity               211
Cloudiness             211
Wind Speed             211
Current Description    211
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                211
City                   211
Country                211
Lat                    211
Lng                    211
Max Temp               211
Humidity               211
Cloudiness             211
Wind Speed             211
Current Description    211
dtype: int64

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Albany,US,81.18,scattered clouds,42.6001,-73.9662,
6,Kapaa,US,84.52,few clouds,22.0752,-159.3190,
12,Gizo,SB,83.79,clear sky,-8.1030,156.8419,
21,Hithadhoo,MV,81.86,overcast clouds,-0.6000,73.0833,
26,Santa Cruz,BO,78.85,few clouds,-17.8000,-63.1667,
29,Port Moresby,PG,80.02,broken clouds,-9.4431,147.1797,
30,Katsuura,JP,81.18,clear sky,35.1333,140.3000,
34,Vilhena,BR,80.82,scattered clouds,-12.7406,-60.1458,
35,Manono,CD,77.29,broken clouds,-7.3000,27.4167,
38,Jamestown,US,80.24,clear sky,42.0970,-79.2353,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Albany,US,81.18,scattered clouds,42.6001,-73.9662,
6,Kapaa,US,84.52,few clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
12,Gizo,SB,83.79,clear sky,-8.1030,156.8419,Imagination Island
21,Hithadhoo,MV,81.86,overcast clouds,-0.6000,73.0833,Scoop Guest House
26,Santa Cruz,BO,78.85,few clouds,-17.8000,-63.1667,Aviador Hotel Boutique
29,Port Moresby,PG,80.02,broken clouds,-9.4431,147.1797,Lamana Hotel (Port Moresby)
30,Katsuura,JP,81.18,clear sky,35.1333,140.3000,Katsuura Hotel Mikazuki
34,Vilhena,BR,80.82,scattered clouds,-12.7406,-60.1458,Hotel Mirage
35,Manono,CD,77.29,broken clouds,-7.3000,27.4167,Hotel hewa bora
38,Jamestown,US,80.24,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))